# Di-Tech Challenge

## Description 

As less than 10% of China’s 1.4 billion citizens own automobiles, the frequency at which Chinese citizens commute on taxis, buses, trains, and planes is the highest in the world. Didi Chuxing, the dominant ride-hailing company in China, processes over 11 million trips, plans over 9 billion routes and collects over 50TB of data per day. To meet needs of riders, Didi must continually innovate to improve cloud computing and big data technologies and algorithms in order to process this massive amount of data and uphold service reliability.
Supply-demand forecasting is critical to enabling Didi to maximise utilisation of drivers and ensure that riders can always get a car whenever and wherever they may need a ride. Supply-demand forecasting helps to predict the volume of drivers and riders at a certain time period in a specific geographic area. For instance, demand tends to surge in residential areas in the mornings and in business districts in the evenings. Supply-demand forecasting allows Didi to predict demand surges and guide drivers to those areas. The end result is higher earnings for drivers and no surge pricing for riders!

In [ ]:
import csv
import sys
from time import time
from datetime import datetime
from itertools import product

import pandas as pd
from pandas import Series, DataFrame

import numpy as np
from numpy import *

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import GradientBoostingRegressor

path = '/Didi_competition/training_data/'

## Order

The Order Info Table shows the basic information of an order, including the passenger and the driver (if driver_id =NULL, it means the order was not answered by any driver), place of origin, destination, price and time. The fields order_id, driver_id, passenger_id, start_hash, and dest_hash are made not sensitive.

In [ ]:
path_order = path + 'order_data/order_data_2016-01-'

header_order = ['order_id','driver_id','passenger_id','start_district_hash',
                  'dest_district_hash', 'Price', 'Time']
                  
list_df_order = [pd.read_csv(path_order + str(i).zfill(2), delimiter='\t', 
                          header=None, names = header_order) for i in xrange(1,4)]

df_order = pd.concat(list_df_order)
df_order.shape
df_order.describe
df_order.head(5)

## Traffic Jam

The Traffic Jam Info Table shows the overall traffic status on the road in a district, including the number of roads at different traffic jam levels in different time periods and different districts. Higher values mean heavier traffic.

In [ ]:
path_traffic = path + 'traffic_data/traffic_data_2016-01-'

header_traffic = ['district_hash','tj_level_1','tj_level_2','tj_level_3',
                  'tj_level_4', 'tj_time']

list_df_traffic = [pd.read_csv(path_traffic + str(i).zfill(2), delimiter='\t', 
                          header=None, names = header_traffic) for i in xrange(1,4)]

df_traffic = pd.concat(list_df_traffic)
df_traffic.shape
df_traffic.describe()
df_traffic.head(4)

## Weather Data

The Weather Info Table shows the weather info every 10 minutes each city. The weather field gives the weather conditions such as sunny, rainy, and snowy etc; all sensitive information has been removed. The unit of temperature is Celsius degree, and PM2.5 is the level of air pollutions.

In [ ]:
path_weather = path + 'weather_data/weather_data_2016-01-'

header_weather = ['Time','Weather','temperature','PM2.5']

list_df_weather = [pd.read_csv(path_weather + str(i).zfill(2), delimiter='\t', 
                          header=None, names = header_weather) for i in xrange(1,4)]

df_weather = pd.concat(list_df_weather)
df_weather.shape
df_weather.describe
df_weather.head(5)

##  District

The District Info Table shows the information about the districts to be evaluated in the contest. You need to do the prediction given the districts from the District Definition Table. In the submission of the results, you need to map the district hash value to district mapped ID.

In [ ]:
path_district = path + 'cluster_map/cluster_map'

header_district = ['district_hash','district_id']

df_district = pd.read_csv(path_district, delimiter='\t', header=None, names = header_district)
df_district.describe()
df_district.head(5)

## POI

The POI Info Table shows the attributes of a district, such as the number of different facilities. For example, 2#1:22 means in this district, there are 22 facilities of the facility class 2#1. 2#1 means the first level class is 2 and the second level is 1, such as entertainment#theater, shopping#home appliance, sports#others. Each class and its number is separated by \t.

In [ ]:
path_POI = path + 'poi_data/poi_data'

header_POI_temp = ['poi']

df_POI_temp = pd.read_csv(path_POI, delimiter='\n', header=None, names = header_POI_temp)

list_POI = df_POI_temp['poi'].str.split('\t').tolist()


`list_POI =[['74c1c25f4b283fa74a5514307b0d0278','1#11:2241','1#10:249','24:1245', ... ],
           ['99c25f4b283fa74a5514307b0d02767h8','20:33449','22:2324','15#24:833', ...],
            ....]`
            
There are 66 disctricts. 26 first level and 26 second level classes.            
            

In [ ]:
complete_list = []
for j in range(66):                
  rec_list =[]
  first_n =[]
  second_n = []
  for i in list_POI[j][1:-1]:
    if '#' in i:
        minilist = map(int, i.replace(':','#').split('#'))
        rec_list.append(minilist)
    else:
        minilist = map(int, ['0']+i.split(':'))
        rec_list.append(minilist) 
    first_n.append(minilist[0])
    second_n.append(minilist[1])             
  complete_list.append(rec_list)


In [ ]:
ids = []
for i in list_POI:
    ids.append(i[0])

my_dict = {}
for i in range(676):
  my_dict[i+1] = [0]*66

# create empty dataframe - discrict_hash as indices
df_POI = pd.DataFrame(my_dict,index=ids)

In [ ]:
n_entries = []
for i in range(66):
     count = 0
     for j in range(676):
         for triplet in complete_list[i]:             
             if triplet[0] == int(floor(j/26)) and triplet[1] == j%26:
                 df_POI[j][i] = triplet[2]
                 #print triplet
                 count = count + 1               
     n_entries.append(count)

df_POI.head(5)    

The notation is the following:

12:234  interpreted as 0#12:234.

1 = 0#1, 2 = 0#2, 3 = 0#3, ...

26=0#26, 27 = 1#1, 28 = 1#2, 29 = 1#3, ...

53 = 2#1, 54 = 2#2, ...

....

..., 676 = 25#26

In [ ]:
# just a test to confirm that it's ok
n_ele = []
for ele in complete_list:
    n_ele.append(len(ele))

array(n_entries)-array(n_ele)

# Some data


There are 

Total orders: 8,540,614 (first 3 days 1,148,684)  
Unique passengers: 1,368,071 (471,999)  
Unique drivers: 93,090 (49,249)  
Duplicate orders: 22,565 [0.26%]  (5,506 [0.48%])  
Orders per passanger: 6.24 (2.43)  
Orders per driver: 91.75 (23.3)  


Orders/day: 406,696 (382,895)


Price  
mean          17.32  
std         16.16  
min          0.0  
25%          8.0  
50%         13.0  
75%         21.0  
max       1731.0  






In [ ]:
df_order[df_order['order_id'] == 'ee99387a4c9abe0698a1c44771ad6b76']

In [ ]:
df_order[df_order['order_id'] == 'ef2fb75c5b051095ea6a440cc5019248']

In [ ]:
# Count how many rows per order_id and driver_id
count = df_order[['order_id', 'driver_id']].groupby('order_id').count()
count.head(5)

In [ ]:
count = count['driver_id']

number_no_picks = sum(count == 0)
number_picks = sum(count == 1)
number_orders = len(count)

print number_no_picks
print number_picks
percentage_no_picks = number_no_picks/number_orders*100


print("There are {} orders-without-drivers out of {} orders: {}.".format(number_no_picks, number_orders, percentage_no_picks))

In [ ]:
#count = count['driver_id']

# Orders picked up by more than one driver?
print(sum(count > 1))
# No more.

# Create gap column
gap = (count == 0).astype('int').tolist()
#df_order['gap'] = gap

#print(df.describe())

In [ ]:
df = df_order

In [ ]:
count = df[['order_id', 'driver_id']].groupby('order_id').count()
count = count['driver_id']

# Proportion of orders not picked up by a driver
s = sum(count == 0)
l = len(count)

print("There are {} orders-without-drivers out of {} orders: {:.1%}.".format(s, l, s/l))
# It appears the gap is simply the number of orders not picked up.

In [ ]:
count = df[['order_id', 'driver_id']].groupby('order_id').count()
count = count['driver_id']
print count

In [ ]:
len(pd.isnull(df).any(1).nonzero()[0])

In [ ]:
dup = df.duplicated(['order_id', 'driver_id', 'passenger_id', 'Time'], keep = 'last')
df = df[~dup]

In [ ]:
# Create gap column
gap = (count == 0).astype('int').tolist()
df['gap'] = gap

#print(df.describe())

#df.drop_duplicates(['order_id', 'driver_id', 'passenger_id', 'time'], keep = 'last')

In [ ]:
# Compute time slot

# Extract the date, and implicitly make the time midnight.
df['Time'] = pd.to_datetime(df.Time)
df['date'] = pd.to_datetime(df.Time.dt.date)
# df['timeonly'] = df.datetime.dt.time

# One day is uniformly divided into 144 ten minute time slots.
df['timeslot'] = (df['Time'] - df['date']).astype('timedelta64[m]')//10

# Drop the time column
# df = df.drop('time', axis = 1)

In [ ]:
# Compute gap per time slot per district
df_select = df[['start_district_hash', 'date', 'timeslot', 'gap']]
df_gap = df_select.groupby(['start_district_hash', 'date', 'timeslot']).sum()

# Flatten data frame after the group by
df_gap = df_gap.reset_index()
print(df_gap.head(2))

# Sanity check: do the numbers add up?
print(sum(df_gap.gap))
# Yup.b

In [ ]:
df_gap_time = df_gap.groupby(['date', 'timeslot']).sum()
df_gap_time.head(5)

In [ ]:
headsize = 800

fig, ax = plt.subplots()
rects1 = ax.plot(df_gap_time['gap'].head(headsize), color='r', alpha=0.5)    
#rects2 = ax.plot(df['EXITSn_hourly'  ].head(headsize), color='b', alpha=0.5)    
#plt.ylabel('ENTRIES / EXITS')
ax.set_yscale('log')   
plt.grid(True)   
plt.show()

## Load Data

In [1]:
##### MAPE

def mape(y_true,y_pred):
    idx = y_true != 0
    return np.mean(np.abs((y_true[idx] - y_pred[idx]) / y_true[idx]))

/Users/enricofonda/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
path = '/Didi_competition/season_2/training_data/'
file_name = 'df_didi_noPOI_mergedleft.pkl'
file_name = 'df_didi_noPOI.pkl'
#df.to_pickle(path + file_name)
df = pd.read_pickle(path + file_name)
df.head(5)


,district_hash,date,timeslot,gap,counter,weekday,district_id,Weather,temperature,PM,tj_level_1,tj_level_2,tj_level_3,tj_level_4
0,08232402614a9b48895cc3d0aeb0e9f2,2016-01-01,6.0,2,4,4,50,1,3.0,177,114,2,1,0
1,08232402614a9b48895cc3d0aeb0e9f2,2016-01-01,7.0,1,1,4,50,1,3.0,177,82,17,10,1
2,08232402614a9b48895cc3d0aeb0e9f2,2016-01-01,8.0,0,1,4,50,1,3.0,177,104,24,2,14
3,08232402614a9b48895cc3d0aeb0e9f2,2016-01-01,14.0,0,1,4,50,1,3.0,206,92,3,0,0
4,08232402614a9b48895cc3d0aeb0e9f2,2016-01-01,15.0,1,1,4,50,1,3.0,206,93,6,4,9


In [3]:
df['timeslot_before'] = df.groupby(['district_id','date']).shift(1).timeslot
df['gap_before'] = df.groupby(['district_id','date']).shift(1).gap
df['counter_before'] = df.groupby(['district_id','date']).shift(1).counter

df['timeslot_before2'] = df.groupby(['district_id','date']).shift(2).timeslot
df['gap_before2'] = df.groupby(['district_id','date']).shift(2).gap
df['counter_before2'] = df.groupby(['district_id','date']).shift(2).counter
df['timeslot_before3'] = df.groupby(['district_id','date']).shift(3).timeslot
df['gap_before3'] = df.groupby(['district_id','date']).shift(3).gap
df['counter_before3'] = df.groupby(['district_id','date']).shift(3).counter

df['delta_timeslot'] = df['timeslot']-df['timeslot_before']
df['delta_timeslot2'] = df['timeslot_before']-df['timeslot_before2']
df['delta_timeslot3'] = df['timeslot_before2']-df['timeslot_before3']

df['tj_1_before'] = df.groupby(['district_id','date']).shift(1).tj_level_1
df['tj_2_before'] = df.groupby(['district_id','date']).shift(1).tj_level_2
df['tj_3_before'] = df.groupby(['district_id','date']).shift(1).tj_level_3
df['tj_4_before'] = df.groupby(['district_id','date']).shift(1).tj_level_4

df['Weather_before'] = df.groupby(['district_id','date']).shift(1).Weather
df['temperature_before'] = df.groupby(['district_id','date']).shift(1).temperature
df['PM_before'] = df.groupby(['district_id','date']).shift(1).PM


#df['Weather_before'].value_counts()
#df['Weather'].value_counts()
#df['Weather_before'] = 'W_' + df['Weather_before'].astype(int).astype(str)


In [4]:
# Deal with NaNs
print df.shape
df = df[np.isfinite(df['timeslot_before'])] # remove NaN rows
df.temperature_before = df.temperature_before.fillna(method='backfill')
df.Weather_before = df.Weather_before.fillna(method='backfill')
print len(df[np.isfinite(df['tj_1_before'])])
print len(df[np.isfinite(df['Weather_before'])])
#df = df[np.isfinite(df['tj_1_before'])]
#df = df[np.isfinite(df['Weather_before'])]
print df.shape

(139582, 33)
138217
138217
(138217, 33)


In [5]:
df=df[df['delta_timeslot'].isin([1.0])]
df=df[df['delta_timeslot2'].isin([1.0])]
df=df[df['delta_timeslot3'].isin([1.0])]
df = df.drop('delta_timeslot', 1)
df = df.drop('delta_timeslot2', 1)
df = df.drop('delta_timeslot3', 1)
print df.shape

(45087, 30)


In [6]:
df['timeslot'] = df['timeslot'].astype(int)
df = df.sort_values(['district_id','date','timeslot'], ascending=[True,True,True])

df.head()

,district_hash,date,timeslot,gap,counter,weekday,district_id,Weather,temperature,PM,...,timeslot_before3,gap_before3,counter_before3,tj_1_before,tj_2_before,tj_3_before,tj_4_before,Weather_before,temperature_before,PM_before
72997,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-01,12,3,73,4,1,1,3.0,206,...,9.0,6.0,120.0,1316.0,241.0,82.0,64.0,1.0,3.0,206.0
72998,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-01,13,1,60,4,1,1,3.0,206,...,10.0,6.0,111.0,1393.0,254.0,65.0,65.0,1.0,3.0,206.0
73002,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-01,20,6,46,4,1,1,3.0,205,...,17.0,20.0,73.0,1211.0,143.0,54.0,51.0,1.0,3.0,205.0
73003,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-01,21,2,41,4,1,1,3.0,205,...,18.0,18.0,65.0,1160.0,163.0,39.0,51.0,1.0,3.0,205.0
73004,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-01,22,1,32,4,1,1,3.0,205,...,19.0,11.0,60.0,1165.0,153.0,48.0,51.0,1.0,3.0,205.0


In [7]:
one_hot_district = pd.get_dummies('D_' + df['district_id'].astype(int).astype(str).apply(lambda x:x.zfill(2)))
df = df.join(one_hot_district)
df.head(50)


one_hot_weekday = pd.get_dummies('day_' + df['weekday'].astype(int).astype(str))
df = df.join(one_hot_weekday)
df.head()
# df = df.drop('day_1',1) - ####################
#df = df.drop('day_3',1)

one_hot_weather = pd.get_dummies(df['Weather_before'])
df = df.join(one_hot_weather)
# df.Weather.unique()
df = df.drop(9.0,1) # drop because Weather 9 not present in the test set.

df.head()

,district_hash,date,timeslot,gap,counter,weekday,district_id,Weather,temperature,PM,...,day_2,day_3,day_4,day_5,day_6,1.0,2.0,3.0,4.0,8.0
72997,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-01,12,3,73,4,1,1,3.0,206,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
72998,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-01,13,1,60,4,1,1,3.0,206,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
73002,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-01,20,6,46,4,1,1,3.0,205,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
73003,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-01,21,2,41,4,1,1,3.0,205,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
73004,90c5a34f06ac86aee0fd70e2adce7d8a,2016-01-01,22,1,32,4,1,1,3.0,205,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [8]:
df = df.drop('district_hash',1)

df = df.drop('tj_level_1',1)
df = df.drop('tj_level_2',1)
df = df.drop('tj_level_3',1)
df = df.drop('tj_level_4',1)
df = df.drop('PM',1)
df = df.drop('temperature',1)
df = df.drop('Weather',1)
df = df.drop('counter',1)
df = df.drop('timeslot_before',1)
df = df.drop('timeslot_before2',1)
df = df.drop('timeslot_before3',1)

df = df.drop('PM_before',1)
df = df.drop('Weather_before',1)

In [9]:
print len(df.columns.tolist())
df.columns.tolist()

93


['date',
 'timeslot',
 'gap',
 'weekday',
 'district_id',
 'gap_before',
 'counter_before',
 'gap_before2',
 'counter_before2',
 'gap_before3',
 'counter_before3',
 'tj_1_before',
 'tj_2_before',
 'tj_3_before',
 'tj_4_before',
 'temperature_before',
 'D_01',
 'D_02',
 'D_03',
 'D_04',
 'D_05',
 'D_06',
 'D_07',
 'D_08',
 'D_09',
 'D_10',
 'D_11',
 'D_12',
 'D_13',
 'D_14',
 'D_15',
 'D_16',
 'D_17',
 'D_18',
 'D_19',
 'D_20',
 'D_21',
 'D_22',
 'D_23',
 'D_24',
 'D_25',
 'D_26',
 'D_27',
 'D_28',
 'D_29',
 'D_30',
 'D_31',
 'D_32',
 'D_33',
 'D_34',
 'D_35',
 'D_36',
 'D_37',
 'D_38',
 'D_39',
 'D_40',
 'D_41',
 'D_42',
 'D_43',
 'D_44',
 'D_45',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'D_50',
 'D_51',
 'D_52',
 'D_53',
 'D_55',
 'D_56',
 'D_57',
 'D_58',
 'D_59',
 'D_60',
 'D_61',
 'D_62',
 'D_63',
 'D_64',
 'D_65',
 'D_66',
 'day_0',
 'day_1',
 'day_2',
 'day_3',
 'day_4',
 'day_5',
 'day_6',
 1.0,
 2.0,
 3.0,
 4.0,
 8.0]

In [10]:
## -- Create training and test set, as two week and 1 week
# if training on full 3 weks: train_variables = df
mask = (df['date'] >= '2016-01-01') & (df['date'] <= '2016-01-14')
#mask_first_week = (df['date'] >= '2016-01-01') & (df['date'] <= '2016-01-07')
train_variables = df.loc[mask]
test_variables = df.loc[~mask]

In [11]:
labels_train = train_variables[['gap']].as_matrix().astype(int)            
#labels_train_orders = train_variables[['counter']].as_matrix().astype(int) 

train_variables = train_variables.drop('date',1)
train_variables = train_variables.drop('district_id',1)
train_variables = train_variables.drop('gap',1)
train_variables = train_variables.drop('weekday',1)


features_train = train_variables.as_matrix().astype(int)
print np.shape(features_train)

(36276, 89)


In [12]:
train_variables.columns.tolist()

['timeslot',
 'gap_before',
 'counter_before',
 'gap_before2',
 'counter_before2',
 'gap_before3',
 'counter_before3',
 'tj_1_before',
 'tj_2_before',
 'tj_3_before',
 'tj_4_before',
 'temperature_before',
 'D_01',
 'D_02',
 'D_03',
 'D_04',
 'D_05',
 'D_06',
 'D_07',
 'D_08',
 'D_09',
 'D_10',
 'D_11',
 'D_12',
 'D_13',
 'D_14',
 'D_15',
 'D_16',
 'D_17',
 'D_18',
 'D_19',
 'D_20',
 'D_21',
 'D_22',
 'D_23',
 'D_24',
 'D_25',
 'D_26',
 'D_27',
 'D_28',
 'D_29',
 'D_30',
 'D_31',
 'D_32',
 'D_33',
 'D_34',
 'D_35',
 'D_36',
 'D_37',
 'D_38',
 'D_39',
 'D_40',
 'D_41',
 'D_42',
 'D_43',
 'D_44',
 'D_45',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'D_50',
 'D_51',
 'D_52',
 'D_53',
 'D_55',
 'D_56',
 'D_57',
 'D_58',
 'D_59',
 'D_60',
 'D_61',
 'D_62',
 'D_63',
 'D_64',
 'D_65',
 'D_66',
 'day_0',
 'day_1',
 'day_2',
 'day_3',
 'day_4',
 'day_5',
 'day_6',
 1.0,
 2.0,
 3.0,
 4.0,
 8.0]

In [ ]:
test_variables = test_variables[test_variables['weekday'].isin([0,2,4,5,6])]
test_variables = test_variables[test_variables['timeslot'].isin([46,58,70,82,94,106,118,130,142])]

results_val = (test_variables['district_id'].astype(str)).reset_index().drop('index',1)
results_val['datetime'] = (test_variables['date'].astype(str) +'-'+ test_variables['timeslot']
                  .astype(int).astype(str)).reset_index().drop('index',1)
 
results_val['gap_outcome'] = test_variables['gap'].reset_index().drop('index',1)
                      

labels_test = test_variables['gap'].as_matrix().astype(int)

test_variables = test_variables.drop('date',1)
test_variables = test_variables.drop('district_id',1)
test_variables = test_variables.drop('gap',1)
test_variables = test_variables.drop('weekday',1)

test_variables.columns.tolist()


features_test = test_variables.as_matrix().astype(int)

In [13]:
np.shape(features_test)

(619, 89)


In [14]:
t0 = time()
clf = GradientBoostingRegressor(n_estimators = 77)
clf = clf.fit(features_train, labels_train)
pred = clf.predict(features_test)
print "training time:", round(time()-t0, 3), "s"

outcome = labels_test.flatten().astype(float)
predict = pred.round().astype(float)#.astype(int)
#predict = ceil(pred.astype(float))#.astype(int)

score = mape(outcome, predict)
print("MAPE: {:.5f}".format(score))

/Users/enricofonda/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training time: 6.29 s
MAPE: 0.49431


In [15]:
predict_test = predict.astype(int)
results_val['pred'] = predict_test
results_val.head()

,district_id,datetime,gap_outcome,pred
0,1,2016-01-15-82,5,8
1,1,2016-01-15-106,12,8
2,1,2016-01-15-142,2,3
3,1,2016-01-16-58,4,4
4,1,2016-01-16-82,2,4


In [ ]:
## - Save to file
path_test = '/Didi_competition/season_2/test_set_2/'
results_val.to_csv(path_test + 'predictions_one-hot-validation-previous.csv',index = False, header = False)

#dict(zip(outcome.tolist(), predict.tolist()))

## Test set

In [ ]:
path_test = '/Didi_competition/season_2/test_set_2/'

file_name = 'df_didi_test_season_2.pkl'
#df.to_pickle(path_test + file_name)
df_test = pd.read_pickle(path_test + file_name)
df_test.head(300)

df_test['date'] = df_test['date'].astype(str)

df_test = df_test.sort_values(['district_id','date','timeslot'], ascending=[True,True,True])

#df_test['Weather','temperature','PM','traffic','tj_level_1','tj_level_2',
#        'tj_level_3','tj_level_4'] = df_test['Weather','temperature','PM',
#        'traffic','tj_level_1','tj_level_2', 'tj_level_3','tj_level_4'
#        ].fillna(method='backfill')
#

In [ ]:
# Create dataframe with the dates and timeslots for which we need the predictions

selected_dates = df_test['date'].unique().tolist()
timeslots = ['46','58','70','82','94','106','118','130','142']

df_temp = pd.DataFrame(list(product(range(1,67),selected_dates,timeslots)), columns=['district_id', 'date','timeslot'])

df_temp['weekday'] = df_temp.date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())#

idx1 =df_temp[(df_temp['date'] == '2016-01-25') & (df_temp['timeslot'] == '46')].index
df_temp = df_temp.drop(idx1)
idx2 =df_temp[(df_temp['date'] == '2016-01-29') & (df_temp['timeslot'] == '46')].index
df_temp = df_temp.drop(idx2)

df_temp['timeslot'] = df_temp['timeslot'].astype(int)

In [ ]:
## Create new dataframe
df_test= pd.merge(df_test, df_temp, on=['date','district_id','timeslot','weekday'], how='outer')
df_test.drop('district_hash',1)

df_test['timeslot'] = df_test['timeslot'].astype(int)
df_test = df_test.sort_values(['district_id','date','timeslot'], ascending=[True,True,True])

In [ ]:
df_test['gap_before'] = df_test.groupby(['district_id','date']).shift(1).gap
df_test['counter_before'] = df_test.groupby(['district_id','date']).shift(1).counter

df_test['gap_before2'] = df_test.groupby(['district_id','date']).shift(2).gap
df_test['counter_before2'] = df_test.groupby(['district_id','date']).shift(2).counter
df_test['gap_before3'] = df_test.groupby(['district_id','date']).shift(3).gap
df_test['counter_before3'] = df_test.groupby(['district_id','date']).shift(3).counter

df_test['tj_1_before'] = df_test.groupby(['district_id','date']).shift(1).tj_level_1
df_test['tj_2_before'] = df_test.groupby(['district_id','date']).shift(1).tj_level_2
df_test['tj_3_before'] = df_test.groupby(['district_id','date']).shift(1).tj_level_3
df_test['tj_4_before'] = df_test.groupby(['district_id','date']).shift(1).tj_level_4

df_test['Weather_before'] = df_test.groupby(['district_id','date']).shift(1).Weather
df_test['temperature_before'] = df_test.groupby(['district_id','date']).shift(1).temperature
df_test['PM_before'] = df_test.groupby(['district_id','date']).shift(1).PM

In [ ]:
# Create dummy variables

one_hot_district = pd.get_dummies('D_' + df_test['district_id'].astype(int).astype(str).apply(lambda x:x.zfill(2)))
df_test = df_test.join(one_hot_district)
# district_id dropped after merging

one_hot_weekday = pd.get_dummies('day_' + df_test['weekday'].astype(int).astype(str))
df_test = df_test.join(one_hot_weekday)
# dropped dropped after merging

one_hot_weather = pd.get_dummies(df_test['Weather_before'])
df_test = df_test.join(one_hot_weather)
df_test = df_test.drop(6.0,1) # dropped because not present in training set
df_test = df_test.drop('Weather_before',1)

df_test.head()

In [ ]:
df_test = df_test.drop('tj_level_1',1)
df_test = df_test.drop('tj_level_2',1)
df_test = df_test.drop('tj_level_3',1)
df_test = df_test.drop('tj_level_4',1)
df_test = df_test.drop('PM',1)
df_test = df_test.drop('temperature',1)
df_test = df_test.drop('Weather',1)
#df_test = df_test.drop('timeslot_before',1)
df_test = df_test.drop('district_hash',1)

df_test = df_test.drop('PM_before',1)

df_test.columns.tolist()

df_test.head()

In [ ]:
df_test = df_test.drop('counter',1)
df_test = df_test.drop('gap',1)

df_test= pd.merge(df_test, df_temp, on=['date','district_id','timeslot','weekday'], how='outer')

df_test['weekday'] = df_test['weekday'].astype(int)
df_test['district_id'] = df_test['district_id'].astype(int)

# define new dataframe - merging 
test_test = pd.merge(df_test, df_temp, on=['date','district_id','timeslot','weekday'], how='inner')
test_test = test_test.sort_values(['date','timeslot','district_id'], ascending=[True,True,True])

In [ ]:
results = (test_test['district_id'].astype(str)).reset_index().drop('index',1)

results['datetime'] = (test_test['date'].astype(str) +'-'+ test_test['timeslot']
                  .astype(int).astype(str)).reset_index().drop('index',1)

test_test = test_test.drop('date',1)
test_test = test_test.drop('weekday',1)
test_test = test_test.drop('district_id',1)

In [ ]:
dict(zip (test_variables.columns.tolist(), test_test.columns.tolist()))

In [ ]:
features_test_test = test_test.as_matrix().astype(int)

In [ ]:
## -- Predictions
pred_test = clf.predict(features_test_test)
predict_test = pred_test.round().astype(int)
results['pred'] = predict_test
results.head()

In [ ]:
## - Save to file
path_test = '/Didi_competition/season_2/test_set_2/'
results.to_csv(path_test + 'predictions-.csv',index = False, header = False)